In [19]:
import os
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import regex as re

In [ ]:
df = pd.read_csv('kickstarter_data_2024.csv')

In [ ]:
blurb_data = df[['blurb', 'state']]


,blurb,state
0,Astrological Connections and Personal Wisdom,live
1,Cozy Kornerz is a fun new redecorating tool th...,live
2,I want to design an exercise Matt with practic...,live
3,A space engineered to promote informed consume...,submitted
4,YourMD Online Connect: Redefining Telehealth f...,submitted


In [ ]:
blurb_data = blurb_data[blurb_data['state'] != "live"]
blurb_data = blurb_data[blurb_data['state'] != "submitted"]
blurb_data = blurb_data[blurb_data['state'] != "canceled"]
blurb_data = blurb_data[blurb_data['state'] != "started"]
blurb_data = blurb_data[blurb_data['state'] != "suspended"]
blurb_data.head()
# blurb_data.reset_index()

,index,blurb,state
0,9,"Small knife. Big tasks. No-frills, purpose dri...",successful
1,10,A beautifully designed stainless steel mirror ...,successful
2,11,A discreet beverage dispensing and storage sol...,successful
3,12,Kickstarter exclusive dark mode hardcover & sp...,successful
4,13,Help us fund the upcoming audiobook by securin...,successful
...,...,...,...
1967818,2207594,"The most comfortable, high quality and eco-fri...",successful
1967819,2207595,Great news for your wardrobe and the planet,successful
1967820,2207596,A collection inspired by vintage Valentine’s l...,successful
1967821,2207598,"""Le Chèile"" is a stunning new album by Mckerro...",successful


In [6]:
blurb_data['state'].value_counts()

state
successful    1259312
failed         708511
Name: count, dtype: int64

In [7]:
blurb_data["state"] = blurb_data["state"].apply(lambda x: 1 if x == "successful" else 0).astype(int)
blurb_data['state'].value_counts()

state
1    1259312
0     708511
Name: count, dtype: int64

In [8]:
blurb_data = blurb_data.drop_duplicates()
blurb_data = blurb_data.reset_index(inplace = False, drop = True)
blurb_data.shape

(207225, 2)

In [9]:
blurb_data[blurb_data.select_dtypes(['object']).columns] = blurb_data.select_dtypes(['object']).astype(str)

In [12]:
def preprocess_text(text):
    # Remove any character that is not a letter (a-z) or white space ( )
    text = re.sub(r'[^a-zA-Z]', " ", text)
    
    # Remove white spaces
    text = re.sub(r'\s+[a-zA-Z]\s+', " ", text)
    text = re.sub(r'\^[a-zA-Z]\s+', " ", text)

    # Multiple white spaces into one
    text = re.sub(r'\s+', " ", text.lower())

    # Remove tags
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", text)

    return text.split()

blurb_data["blurb"] = blurb_data["blurb"].apply(preprocess_text)
blurb_data.head()

,blurb,state
0,"[small, knife, big, tasks, no, frills, purpose...",1
1,"[a, beautifully, designed, stainless, steel, m...",1
2,"[a, discreet, beverage, dispensing, and, stora...",1
3,"[kickstarter, exclusive, dark, mode, hardcover...",1
4,"[help, us, fund, the, upcoming, audiobook, by,...",1


In [ ]:
download("wordnet")
lemmatizer = WordNetLemmatizer()

download("stopwords")
stop_words = stopwords.words("english")

remove_words = ['designed']

stop_words.append(remove_words)

[nltk_data] Downloading package wordnet to C:\Users\Philip
[nltk_data]     Nancollas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Philip
[nltk_data]     Nancollas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def lemmatize_text(words, lemmatizer = lemmatizer):
    tokens = [lemmatizer.lemmatize(word) for word in words]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [word for word in tokens if len(word) > 3]
    return tokens

blurb_data["blurb"] = blurb_data["blurb"].apply(lemmatize_text)
blurb_data.head()

In [ ]:
success_set = blurb_data[blurb_data['state'] == 1]
failed_set = blurb_data[blurb_data['state'] == 0]